In [1]:
import sys
sys.path.insert(0, "../..")
def set_size(width = "thesis", fraction=1, subplots=(1, 1)):
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('seaborn')

from os import listdir
from os.path import isfile, join

from module.base.network import Network
from module.simulation.meanfield import MeanField
import module.components.CONST as CONST
from module.simulation.set_meanfield2 import SetMeanField2
from module.simulation.masterequation import MasterEquation
from module.simulation.quick_meanfield2 import QuickMeanField2
from module.simulation.meanfield2 import MeanField2
from module.components.lawrence_dist import LawrenceDist
from module.components.discrete_gaussian1D import DiscreteGaussian1D

from module.components.restricted_gaussian1D import RestrictedGaussian1D
from module.components.p2_4dist import p2_4dist

/tmp/ipykernel_620/2397363.py:30: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
def calc_rel_freq(data):
    data = data.round()
    phase_space = np.arange(data.min(), data.max() + 1)
    n = []
    N = data.shape[0]
    for i, val in enumerate(phase_space):
        n.append(np.where(data == phase_space[i])[0].shape[0])

    n = np.array(n)
    probs = n/N

    return phase_space, probs

In [3]:
ld = LawrenceDist()
p2dist = p2_4dist()

In [4]:
i = 25 # voltage config
island =  3 # island
voltage_config = np.loadtxt("../../data/cont_comparison/voltage_configs.csv")[i]
kmc_states = np.loadtxt("../../data/cont_comparison/kmc/"+str(i)+"_.csv")
kmc_phase_space, probs_kmc = calc_rel_freq(kmc_states[:, island])
net = Network(4,4,1,[[0,0,0],[3,0,0],[0,3,0],[3,3,0]])
net.set_voltage_config(voltage_config[0:4], 0)

mf = MeanField(net)
mf2 = MeanField2(net, include_covs = True)
mf2_no_cov = MeanField2(net, include_covs = False)
qmf2 = QuickMeanField2(net)

In [6]:
means = np.zeros(16)
qmf2.numeric_integration_solve(N = 0)
qmf2.ADAM_solve(N = 0)

In [7]:
mf2.ADAM_solve(N = 0)

In [7]:
# first order
means = mf.numeric_integration_solve(means, verbose = True)

convergence: 4.7697372804411664e-05


In [12]:
# full second order

mf2.ADAM_solve(verbose = True, N = 30, dt = 0.1, reset = False)

Adam convergence: (0.0856166319243797, 0.05526217420617863, 0.038918955368433296)


In [14]:
# second order cov = 0

mf2_no_cov.ADAM_solve(verbose = True, N = 80)

Adam convergence: (0.08240215965229931, 0.07564171146987231, 0.0)


In [11]:
# p^2-fdist

#qmf2.numeric_integration_solve(verbose = True, dt = 0.1, reset = False)
qmf2.ADAM_solve(N = 300, verbose = True, reset = False, learning_rate = 0.1)

ADAM convergence: (0.010967650957424403, 0.0435144570570349)
